In [ ]:
!python3 etl.py

In [ ]:
import os
import configparser
from pyspark.sql import SparkSession

In [ ]:
config = configparser.ConfigParser()
config.read('dl.cfg', encoding='utf-8-sig')

os.environ['AWS_ACCESS_KEY_ID']=config['AWS']['AWS_ACCESS_KEY_ID']
os.environ['AWS_SECRET_ACCESS_KEY']=config['AWS']['AWS_SECRET_ACCESS_KEY']

SOURCE_S3_BUCKET = config['S3']['SOURCE_S3_BUCKET']
DEST_S3_BUCKET = config['S3']['DEST_S3_BUCKET']

In [ ]:
spark = SparkSession \
        .builder \
        .config("spark.jars.packages", "org.apache.hadoop:hadoop-aws:2.7.0") \
        .getOrCreate()

In [ ]:
hadoop_conf = spark.sparkContext._jsc.hadoopConfiguration()
#hadoop_conf.set("fs.s3a.aws.credentials.provider", "org.apache.hadoop.fs.s3a.TemporaryAWSCredentialsProvider")
hadoop_conf.set("fs.s3a.access.key", os.environ['AWS_ACCESS_KEY_ID'])
hadoop_conf.set("fs.s3a.secret.key", os.environ['AWS_SECRET_ACCESS_KEY'])
#hadoop_conf.set("fs.s3a.session.token", os.environ['AWS_SESSION_TOKEN'])

In [ ]:
df = spark.read.parquet(DEST_S3_BUCKET + 'songplays/year=2018/month=11/*.parquet')

In [ ]:
df.head(5)